### Instruction finetuning using SFT

In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
# from peft import LoraConfig, PeftConfig
from trl import SFTTrainer  #SFTConfig, SFTTrainer
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-22 14:49:06.862019: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 14:49:07.918104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /anaconda/envs/azureml_py38_PT_TF/lib/
2024-05-22 14:49:07.918239: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin

In [2]:
from peft import LoraConfig, PeftConfig

In [3]:
import trl

In [4]:
df = pd.read_csv('./all-data.csv',
                 names=["sentiment","text"],
                 encoding='utf-8',
                 encoding_errors="replace")

In [5]:
df

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [6]:
df.sentiment.unique().tolist()

['neutral', 'negative', 'positive']

In [7]:
X_train, X_test = [],[]

for sentiment in df.sentiment.unique().tolist():
    sample_train, sample_test = train_test_split(df[df.sentiment==sentiment],
                                                 train_size=300,
                                                 test_size=300,
                                                 random_state=100)
    
    X_train.append(sample_train)
    X_test.append(sample_test)

In [8]:
len(X_train)

3

In [9]:
X_train_df = pd.concat(X_train).sample(frac=1, random_state=10)

X_test_df = pd.concat(X_test).sample(frac=1, random_state=10)

In [10]:
X_train_df.shape, X_test_df.shape

((900, 2), (900, 2))

In [11]:
X_train_df.index.min(), X_test_df.index.min()

(2, 7)

In [12]:
X_train_df

,sentiment,text
671,negative,Result before taxes decreased to nearly EUR 14...
2869,neutral,Den Bosch-based TomTom is Europe 's largest ma...
1763,positive,The concept enables a commercially affordable ...
1984,neutral,"The five-storey , eco-efficient building will ..."
2031,neutral,"Operating income was EUR -24.9 mn , including ..."
...,...,...
4396,negative,EMSA Deputy Chairman of the Board Juri Lember ...
4803,negative,"UPM-Kymmene Corp. , the world 's largest maker..."
4624,negative,"At the close , the OMX Helsinki 25 was 0.01 pc..."
1633,neutral,The first charging stations will be made avail...


In [13]:
# Generating eval set:

idx_for_eval = [idx for idx in df.index if idx not in list(X_train_df.index) + list(X_test_df.index)]
idx_for_eval[0:10]


[0, 1, 3, 4, 5, 6, 8, 9, 10, 11]

In [14]:
X_eval_df = df[df.index.isin(idx_for_eval)]

X_eval_df.shape

(3046, 2)

In [15]:
X_eval_df

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
5,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...
4819,neutral,"Nevertheless , the development can not be allo..."
4820,neutral,"Besides , as there is no depositor preference ..."
4823,neutral,It holds 38 percent of Outokumpu 's shares and...
4826,neutral,Mobile communication and wireless broadband pr...


In [16]:
X_train_df.reset_index(inplace=True, drop=True)
X_test_df.reset_index(inplace=True, drop=True)
X_eval_df.reset_index(inplace=True, drop=True)


In [17]:
def prompt_for_training(x):
    prompt = f""" Find the correct sentiment for the phrase: {x['text']} is:
    \n\n Positive
    \n Negative
    \n Neutral
    \n Cannot be determined
    \n\nSolution: The correct option is {x['sentiment']}
    """.strip()
    return prompt

In [18]:
print(prompt_for_training(X_train_df.iloc[0,:]))

Find the correct sentiment for the phrase: Result before taxes decreased to nearly EUR 14.5 mn , compared to nearly EUR 20mn in the previous accounting period . is:
    

 Positive
    
 Negative
    
 Neutral
    
 Cannot be determined
    

Solution: The correct option is negative


In [19]:
X_train_prompt = pd.DataFrame(X_train_df.apply(lambda x: prompt_for_training(x), axis=1),
                              columns=['text'])
X_train_prompt

,text
0,Find the correct sentiment for the phrase: Res...
1,Find the correct sentiment for the phrase: Den...
2,Find the correct sentiment for the phrase: The...
3,Find the correct sentiment for the phrase: The...
4,Find the correct sentiment for the phrase: Ope...
...,...
895,Find the correct sentiment for the phrase: EMS...
896,Find the correct sentiment for the phrase: UPM...
897,Find the correct sentiment for the phrase: At ...
898,Find the correct sentiment for the phrase: The...


In [20]:
X_train_prompt[0:1].values

array([['Find the correct sentiment for the phrase: Result before taxes decreased to nearly EUR 14.5 mn , compared to nearly EUR 20mn in the previous accounting period . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is negative']],
      dtype=object)

In [21]:
X_eval_prompt = pd.DataFrame(X_eval_df.apply(lambda x: prompt_for_training(x), axis=1),
                              columns=['text'])
X_eval_prompt

,text
0,Find the correct sentiment for the phrase: Acc...
1,Find the correct sentiment for the phrase: Tec...
2,Find the correct sentiment for the phrase: Wit...
3,Find the correct sentiment for the phrase: Acc...
4,Find the correct sentiment for the phrase: FIN...
...,...
3041,Find the correct sentiment for the phrase: Nev...
3042,Find the correct sentiment for the phrase: Bes...
3043,Find the correct sentiment for the phrase: It ...
3044,Find the correct sentiment for the phrase: Mob...


In [22]:
def prompt_for_testing(x):
    prompt = f""" Find the correct sentiment for the phrase: {x['text']} is:
    \n\n Positive
    \n Negative
    \n Neutral
    \n Cannot be determined
    \n\nSolution: The correct option is""".strip()
    return prompt

In [23]:
X_test_prompt = pd.DataFrame(X_test_df.apply(lambda x: prompt_for_testing(x), axis=1),
columns=['text'])
X_test_prompt

,text
0,Find the correct sentiment for the phrase: Sal...
1,Find the correct sentiment for the phrase: The...
2,Find the correct sentiment for the phrase: Our...
3,Find the correct sentiment for the phrase: Aff...
4,Find the correct sentiment for the phrase: Bot...
...,...
895,Find the correct sentiment for the phrase: Ope...
896,Find the correct sentiment for the phrase: The...
897,Find the correct sentiment for the phrase: ( A...
898,Find the correct sentiment for the phrase: Rus...


In [24]:
X_test_prompt[0:1]['text'].values

array(["Find the correct sentiment for the phrase: Salonen added that data shows producers ' pulp inventories in North America are declining . ' is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is"],
      dtype=object)

In [25]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

In [26]:
compute_dtype = getattr(torch, "float16")

In [27]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
# )

In [28]:
torch.__version__

'2.3.0+cu121'

In [29]:
transformers.__version__

'4.40.2'

In [30]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    # quantization_config=bnb_config, 
)


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.15s/it]


In [31]:
# import accelerate

In [32]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [33]:
max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                          max_seq_length=max_seq_length,
                                         )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
tokenizer.eos_token

'<|endoftext|>'

In [35]:
tokenizer.pad_token = tokenizer.eos_token

In [36]:
def predict(X_test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        
        pipe = pipeline(task="text-generation", 
                                model=model, 
                                tokenizer=tokenizer,
                                max_new_tokens = 3, 
                                temperature = 0.0,
                            )
        prompt = X_test.iloc[i]["text"]
        result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
        answer = result[0]['generated_text'].split("The correct option is")[-1].lower()

        if "positive" in answer:
                    y_pred.append("positive")
        elif "negative" in answer:
                    y_pred.append("negative")
        elif "neutral" in answer:
                    y_pred.append("neutral")
        else:
                    y_pred.append("none")
    return y_pred
                    

In [37]:
X_test_prompt.shape

(900, 1)

In [38]:
X_test_prompt[0:10]

,text
0,Find the correct sentiment for the phrase: Sal...
1,Find the correct sentiment for the phrase: The...
2,Find the correct sentiment for the phrase: Our...
3,Find the correct sentiment for the phrase: Aff...
4,Find the correct sentiment for the phrase: Bot...
5,Find the correct sentiment for the phrase: The...
6,Find the correct sentiment for the phrase: The...
7,Find the correct sentiment for the phrase: Rev...
8,Find the correct sentiment for the phrase: In ...
9,Find the correct sentiment for the phrase: Ope...


In [39]:
y_pred = predict(X_test_prompt[0:50], model, tokenizer)

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0/50 [00:00<?, ?it/s]/

In [40]:
y_pred

['neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive']

In [41]:
def evaluate(y_true, y_pred):
    labels = ['positive', 'neutral', 'negative']
    mapping = {'positive': 2, 'neutral': 1, 'none':1, 'negative': 0}
    def map_func(x):
        return mapping.get(x, 1)
    
    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')
    
    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels
    
    for label in unique_labels:
        label_indices = [i for i in range(len(y_true)) 
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')
        
    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)
    
    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1, 2])
    print('\nConfusion Matrix:')
    print(conf_matrix)



In [42]:
y_true = X_test_df.sentiment

In [43]:
# no finetuning #zero-shot
evaluate(y_true[0:50], y_pred)

Accuracy: 0.780
Accuracy for label 0: 0.800
Accuracy for label 1: 0.800
Accuracy for label 2: 0.733

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        15
           1       0.73      0.80      0.76        20
           2       0.69      0.73      0.71        15

    accuracy                           0.78        50
   macro avg       0.80      0.78      0.79        50
weighted avg       0.80      0.78      0.78        50


Confusion Matrix:
[[12  2  1]
 [ 0 16  4]
 [ 0  4 11]]


In [44]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules="all-linear",
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM",
)

In [45]:
# training_arguments = TrainingArguments(
#     output_dir="logs",
#     num_train_epochs=2,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8, # 4
#     # optim="paged_adamw_32bit",
#     save_steps=0,
#     logging_steps=25,
#     learning_rate=2e-4,
#     weight_decay=0.001,
#     # fp16=True,
#     bf16=False,
#     max_grad_norm=0.3,
#     max_steps=-1,
#     warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="cosine",
#     # report_to="tensorboard",
#     evaluation_strategy="epoch",
#     no_cuda=True
# )

In [60]:
training_arguments = TrainingArguments(
        num_train_epochs=8,
        output_dir="./results",
        # bf16=True, #change for CPU
        # use_ipex=True, #change for CPU IPEX
        no_cuda=True,
        fp16_full_eval=False,
        # warmup_ratio=0.03,
        # learning_rate=2e-4,
        # weight_decay=0.001,
        # lr_scheduler_type="cosine",
        # evaluation_strategy="epoch"
    )

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/transformers/training_args.py:1453: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [61]:
train_data = Dataset.from_pandas(X_train_prompt)
eval_data = Dataset.from_pandas(X_eval_prompt)

In [62]:
list(train_data)[0:10]

[{'text': 'Find the correct sentiment for the phrase: Result before taxes decreased to nearly EUR 14.5 mn , compared to nearly EUR 20mn in the previous accounting period . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is negative'},
 {'text': "Find the correct sentiment for the phrase: Den Bosch-based TomTom is Europe 's largest maker of automotive navigation devices , while Cayman Islands-based Garmin is larger in the U.S. and overall . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is neutral"},
 {'text': 'Find the correct sentiment for the phrase: The concept enables a commercially affordable way to manufacture high-quality TCO coated glass for the solar industry . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is positive'},
 {'text': 'Find the correct sent

In [63]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0/900 [00:00<?, ? examples/s]

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3046/3046 [00:00<00:00, 25313.00 examples/s]


In [64]:
list(eval_data)

[{'text': 'Find the correct sentiment for the phrase: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is neutral'},
 {'text': 'Find the correct sentiment for the phrase: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said . is:\n    \n\n Positive\n    \n Negative\n    \n Neutral\n    \n Cannot be determined\n    \n\nSolution: The correct option is neutral'},
 {'text': 'Find the correct sentiment for the phrase: With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability . is:\n    \n\n Positive\n    \n Negativ

In [ ]:
# How to put batch_size here ?
# How to print loss for each epoch or per 100 step ?

In [65]:
# Train model
trainer.train()

Step,Training Loss
500,1.056300


/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=904, training_loss=0.9130189608683629, metrics={'train_runtime': 33904.4344, 'train_samples_per_second': 0.212, 'train_steps_per_second': 0.027, 'total_flos': 1.786594903842816e+16, 'train_loss': 0.9130189608683629, 'epoch': 8.0})

In [68]:
# Save trained model
trainer.model.save_pretrained("./phi3/finetuned_phi3_sentiment")

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [58]:
# 2 epochs
y_pred = predict(X_test_prompt[0:50], model, tokenizer)
evaluate(y_true[0:50], y_pred)

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0/50 [00:00<?, ?it/s]/

Accuracy: 0.800
Accuracy for label 0: 0.800
Accuracy for label 1: 0.800
Accuracy for label 2: 0.800

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        15
           1       0.80      0.80      0.80        20
           2       0.67      0.80      0.73        15

    accuracy                           0.80        50
   macro avg       0.82      0.80      0.81        50
weighted avg       0.82      0.80      0.80        50


Confusion Matrix:
[[12  1  2]
 [ 0 16  4]
 [ 0  3 12]]


In [59]:
y_pred

['positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive']

In [69]:
# 8 epochs
y_pred = predict(X_test_prompt[0:100], model, tokenizer,)
evaluate(y_true[0:100], y_pred)

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/100 [00:00<?, ?it/s]/

Accuracy: 0.860
Accuracy for label 0: 0.938
Accuracy for label 1: 0.944
Accuracy for label 2: 0.688

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        32
           1       0.76      0.94      0.84        36
           2       0.88      0.69      0.77        32

    accuracy                           0.86       100
   macro avg       0.88      0.86      0.86       100
weighted avg       0.87      0.86      0.86       100


Confusion Matrix:
[[30  1  1]
 [ 0 34  2]
 [ 0 10 22]]


In [67]:
y_pred

['positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive']